In [1]:
!pip install --user DyMat

In [2]:
import pandas as pd
import numpy as np
import DyMat as DM
from glob import glob

import matplotlib.pyplot as plt

import bokeh.plotting as bkplt
import bokeh.models as bkmdl
import bokeh.layouts as bklay
import bokeh.palettes as bkpalet #from bokeh.palettes import PuOr
import bokeh.transform as bktrans # from bokeh.transform import cumsum

In [3]:
def bokeh_multilines(dict_source, columns2plot, index_col, plot_h, plot_w, x_ax_name, y_ax_name, plt_title):
    
    source = bkmdl.ColumnDataSource(dict_source)
    
    p = bkplt.figure(x_axis_label=x_ax_name,
                     y_axis_label = y_ax_name,
                     #x_axis_type= 'datetime',
                     plot_height = plot_h, 
                     plot_width=plot_w,
                    title = plt_title)
    
    plots = []
    #colors = ['orange','gray','green','pink','purple','red','cyan','black','blue','yellow','']
    colors = (['orange',
                'gray',
                'Blue',
                'BlueViolet', 
                'Brown', 
                'CadetBlue', 
                'Chartreuse', 
                'Chocolate', 
                'CornflowerBlue', 
                'Cornsilk', 
                'Crimson', 
                'Cyan', 
                'DarkBlue', 
                'DarkCyan', 
                'DarkGoldenRod']) 
    for ii in range(len(columns2plot)):
        plots.append( p.line(x=index_col,y=columns2plot[ii],color=colors[ii], source=source, line_width=1) )

        
    legend = bkmdl.Legend(items=[(leg,[plt]) for leg,plt in zip(columns2plot,plots)], location="center")
    p.add_layout(legend, 'right')

    p.legend.location = "top_left" 
    p.legend.click_policy="hide"
        
    p.yaxis.formatter = bkmdl.BasicTickFormatter(use_scientific = False)
    
    return p

In [4]:
def bokeh_multiax(dict_source, columns_ax1, columns_ax2, index_col, plot_h, plot_w, x_ax_name, y_ax1_name, y_ax2_name, plt_title):
    
    source = bkmdl.ColumnDataSource(dict_source)
    
    p = bkplt.figure(title = plt_title,
                     x_axis_label = x_ax_name,
                     y_axis_label = y_ax1_name,
                     #x_axis_type= 'datetime',
                     plot_height = plot_h, 
                     plot_width = plot_w
                    )
    
    legends = []
    plts2leg = []
    
    colors = (['orange',
                'gray',
                'Blue',
                'BlueViolet', 
                'Brown', 
                'CadetBlue', 
                'Chartreuse', 
                'Chocolate', 
                'CornflowerBlue', 
                'Cornsilk', 
                'Crimson', 
                'Cyan', 
                'DarkBlue', 
                'DarkCyan', 
                'DarkGoldenRod'])
    
    # Bokeh plot
    y_overlimit = 0.05 # show y axis below and above y min and max value

    # FIRST AXIS
    min_y1,max_y1 = ( np.array([np.array(dict_source[columns_ax1[ii]]).min() for ii in range(len(columns_ax1))]).min(),
                      np.array([np.array(dict_source[columns_ax1[ii]]).max() for ii in range(len(columns_ax1))]).max() )
    
    for ii in range(len(columns_ax1)):
        plt_tmp = p.line(x=index_col, y=columns_ax1[ii], source=source, line_width=1, color=colors[ii])
        legends.append(columns_ax1[ii])
        plts2leg.append(plt_tmp)
    p.y_range = bkmdl.Range1d(min_y1*(1-y_overlimit), max_y1*(1+y_overlimit))
    ii_colorshift = ii + 1
    
    #bkplt.output_file('./bokeh_res/yaquekea.html', title='hola')
    #bkplt.save(p)
    
    # SECOND AXIS
    min_y2,max_y2 = ( np.array([np.array(dict_source[columns_ax2[ii]]).min() for ii in range(len(columns_ax2))]).min(),
                      np.array([np.array(dict_source[columns_ax2[ii]]).max() for ii in range(len(columns_ax2))]).max() )
    #print([np.array(dict_source[columns_ax2[ii]]).max() for ii in range(len(columns_ax2))])
    y2_colrange = 'y2_colrange'
    p.extra_y_ranges = {
        y2_colrange: bkmdl.Range1d(min_y2*(1-y_overlimit), max_y2*(1+y_overlimit))
    }
    #print(min_y2,max_y2)
    p.add_layout(bkmdl.LinearAxis(y_range_name=y2_colrange,axis_label=y_ax2_name), "right")

    for ii in range(len(columns_ax2)):
        plt_tmp = p.line(x=index_col, y=columns_ax2[ii], source=source, y_range_name=y2_colrange, line_width=2, color=colors[ii_colorshift + ii])
        legends.append(columns_ax2[ii])
        plts2leg.append(plt_tmp)
        
    legend = bkmdl.Legend(items=[(leg,[plt]) for leg,plt in zip(legends,plts2leg)], location="center")
    p.add_layout(legend, 'right')
    
    p.xaxis.major_label_orientation = np.pi/4
    
    return p

In [10]:
matfilename = '/home/karbo/CSP/tower_two_tanks/SolarTherm.SaltTwoTanks/SolarTherm.SaltTwoTanks_res.mat'

In [11]:
#n_res = 3

#d = DM.DyMatFile('../csp_torre_base/csp_torre_res_%d.mat'%(n_res))
d = DM.DyMatFile(matfilename)

var_names1 = list(d.names(1))
var_names1.sort()
var_names2 = list(d.names(2))
var_names2.sort()
#print('block1')
#print(var_names1)
#print('block2')
#print(var_names2)
#print(d.nameTree())
data_cols = ['P_elec','E_elec',
             'receiver.Q_rcv',
             'tankHot.W_net','tankCold.W_net',
             'heliostatsField.heat.Q_flow',
             'controlCold.L_mea','controlHot.L_mea']

data_arr = d.getVarArray(data_cols)
data_df = pd.DataFrame(data_arr).T
data_df.columns = ['time']+data_cols
data_df = data_df.set_index('time',False)
#print(data_df[data_cols])
#plt.figure(figsize=(50,5))
#plt.plot(data_df['E_elec'])
#plt.figure(figsize=(50,5))
#plt.plot(data_df['P_elec'])

In [12]:
matfiles = glob('../csp_torre_base/csp_torre_res*.mat')
data_cols = ['P_elec','E_elec',
             'receiver.Q_rcv',
             'tankHot.W_net','tankCold.W_net',
             'heliostatsField.heat.Q_flow',
             'controlCold.L_mea','controlHot.L_mea']
param_cols = ['t_storage','hot_tnk_empty_lb','hot_tnk_empty_ub']

In [13]:
#data_df
plot_loc_ruta_base = './'

nombreplot = 'plot_base_res_1.html'

data_dict = data_df.iloc[:].to_dict(orient='list')

#columns2plot = ['tankHot.W_net','tankCold.W_net']
columns2plot = ['P_elec']
#columns2plot = ['E_elec']
index_col = 'time'
plot_h = 500
plot_w = 1500
x_ax_name = 'segundos'
y_ax_name = 'power'
plt_title = 'ploto'
p = bokeh_multilines(data_dict,
                     columns2plot,
                     index_col,
                     plot_h,plot_w,
                     x_ax_name,y_ax_name,
                     plt_title)

bkplt.output_file(plot_loc_ruta_base+nombreplot, title='hola')
bkplt.save(p)

'/home/karbo/CSP/sunaicl-solartherm/SolarTherm/new_feature_functions/plot_base_res_1.html'

In [ ]:
plot_loc_ruta_base = './bokeh_res/'

#columns2plot = ['tankHot.W_net','tankCold.W_net']
columns_ax1 = ['P_elec']
columns_ax2 = ['controlCold.L_mea','controlHot.L_mea']
index_col = 'time'
plot_h = 300
plot_w = 1000
x_ax_name = 'segundos'
y_ax1_name = 'power_e'
y_ax2_name = 'power_t'
plt_title = 'ploto'

max_energias = []
for matfile in matfiles:
    print(matfile)
    d = DM.DyMatFile(matfile)
    param_arr = d.getVarArray(param_cols)
    data_arr = d.getVarArray(data_cols)
    data_df = pd.DataFrame(data_arr).T
    data_df.columns = ['time']+data_cols
    data_df = data_df.set_index('time',False)
    
    data_dict = data_df.iloc[:].to_dict(orient='list')
    
    p = bokeh_multiax(data_dict,
                      columns_ax1,
                      columns_ax2,
                      index_col,
                      plot_h,plot_w,
                      x_ax_name,y_ax1_name,y_ax2_name,
                      plt_title)
    nombreplot = 'res_t-sto%d_ht-tnk-mpty_lb%d_tnk-mpty_ub%d.html'%(int(param_arr[1,0]),
                                                                    int(param_arr[2,0]),
                                                                    int(param_arr[3,0]))
    max_energias.append(data_df['E_elec'].max())
    bkplt.output_file(plot_loc_ruta_base+nombreplot, title='hola')
    bkplt.save(p)
    print(plot_loc_ruta_base+nombreplot)
    

In [ ]:
np.where(np.array(max_energias) == np.amax(np.array(max_energias)))

In [ ]:
2.77778e-7*np.amin(np.array(max_energias))